## Web Scraping Watch Product in *jamtangan.com* with Python

### Import Necessary Libraries

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchWindowException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import pandas as pd
import time
import json
import csv
import traceback

from email.message import EmailMessage
import ssl, smtplib

### Function Definition

#### Supporting Function

In [6]:
def to_isoformat(input):
    month_dict = {
        "Jan": "01",
        "Feb": "02",
        "Mar": "03",
        "Apr": "04",
        "Mei": "05",
        "Jun": "06",
        "Jul": "07",
        "Ags": "08",
        "Sep": "09",
        "Okt": "10",
        "Nov": "11",
        "Des": "12"
    }

    for key, value in month_dict.items():
        input = input.replace(key, value)
    
    datetime_object = datetime.strptime(input, "%d %m %Y, %H:%M WIB")

    date = datetime_object.date()
    time = datetime_object.time()

    return [date, time]

def to_email(name):
    name = name.lower().strip()
    email = name.replace(" ", "") + "@gmail.com"
    return email

def csv_to_json(csv_file_path, json_file_path):
    json_array = []

    with open(csv_file_path, encoding='utf-8') as csvf: 
        csv_reader = csv.DictReader(csvf) 

        for row in csv_reader: 
            json_array.append(row)

    with open(json_file_path, 'w', encoding='utf-8') as jsonf: 
        json_string = json.dumps(json_array, indent=4)
        jsonf.write(json_string)

#### Extract Product Table Function

In [7]:
def extract_product(driver, wait):
    product_dict = {
        "Brand" : "",
        "Model No" : "",
        "Series" : "",
        "Gender" : "",
        "Colour" : "",
        "Luminous" : "",
        "Calendar" : "",
        "Water Resistant" : "",
        "Case Diameter (mm)" : "",
        "Strap Material" : "",
    }

    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".tab-content .grid")))
    spec_grid = driver.find_element(By.CSS_SELECTOR, ".tab-content .grid")
    spec_list = spec_grid.find_elements(By.CSS_SELECTOR, ".spec-item")
    
    for spec in spec_list:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".leading-6")))
        spec_title = spec.find_element(By.CSS_SELECTOR, ".font-black").text
        data = spec.find_elements(By.CSS_SELECTOR, ".leading-6")
        if spec_title in product_dict.keys():
            key = data[0].text.strip()
            value = data[1].text.strip()
            product_dict[key] = value
        elif (spec_title == "Case Diameter"):
            value = float(data[1].text.replace("mm", "").strip())
            product_dict["Case Diameter (mm)"] = value
    
    for (key, value) in product_dict.items():
        if value == "": product_dict[key] = None

    return product_dict

#### Extract Sales Table Function

In [8]:
def extract_sales(driver, wait):
    sales_dict = {
        "Product Name" : "",
        "Brand" : "",
        "Model No" : "",
        "Normal Price" : "",
        "Discounted Price" : "",
        "Discount Percentage" : "",
        "Number of Seen" : "",
        "Number of Sold" : "",
        "Offline Stock Status" : "",
        "Online Stock Status" : "",
    }

    spec_grid = driver.find_element(By.CSS_SELECTOR, ".tab-content .grid")
    spec_list_raw = spec_grid.find_elements(By.CSS_SELECTOR, ".spec-item")
    for spec in spec_list_raw:
        if spec.find_element(By.CSS_SELECTOR, ".font-black").text in ["Brand", "Model No"]:
            data = spec.find_elements(By.CSS_SELECTOR, ".leading-6")
            key = data[0].text.strip()
            value = data[1].text.strip()
            sales_dict[key] = value

    sales_dict["Product Name"] = driver.find_element(By.TAG_NAME, "h1").text.strip()

    try:
        normal_price = driver.find_element(By.CSS_SELECTOR, "div[data-testid='test-product-info'] .line-through").text.strip()
    except:
        normal_price = driver.find_element(By.CSS_SELECTOR, "div[data-testid='test-product-info'] .text-xl").text.strip()
    discounted_price = driver.find_element(By.CSS_SELECTOR, "div[data-testid='test-product-info'] .text-xl").text.strip()
    sales_dict["Normal Price"] = int(normal_price.replace("Rp", "").replace(".", "").strip())
    sales_dict["Discounted Price"] = int(discounted_price.replace("Rp", "").replace(".", "").strip())

    discount_percentage = (sales_dict["Normal Price"] - sales_dict["Discounted Price"]) / sales_dict["Normal Price"]
    sales_dict["Discount Percentage"] = round(discount_percentage * 100, 2)

    num_seen = driver.find_element(By.CSS_SELECTOR, ".ic-eye + div > .text-sm").text.strip()
    if (num_seen.__contains__("Rb")):
        num_seen = float(num_seen.replace(" Rb", "").strip()) * 1000
    sales_dict["Number of Seen"] = int(num_seen)

    num_sold = driver.find_element(By.CSS_SELECTOR, ".ic-cart.mr-1 + div > .text-sm").text.strip()
    if (num_sold.__contains__("Rb")):
        num_sold = float(num_sold.replace(" Rb", "").strip()) * 1000
    sales_dict["Number of Sold"] = int(num_sold)

    try:
        empty_badge = driver.find_element(By.CSS_SELECTOR, ".badge.bg-accent-red")
        if (empty_badge != None and empty_badge.text.strip().__contains__("habis")):
            sales_dict["Online Stock Status"] = "Not Available"
        else:
            raise Exception("The badge is not empty badge")
    except:
        online_stock_status = driver.find_element(By.CSS_SELECTOR, ".stepper-wrapper + div").text.strip()
        if (online_stock_status == "STOK ONLINE < 5 PCS"): sales_dict["Online Stock Status"] = "Low (< 5 PCS)"
        elif (online_stock_status == "STOK ONLINE > 5 PCS"): sales_dict["Online Stock Status"] = "High (>= 5 PCS)"
        else: sales_dict["Online Stock Status"] = "Unknown"

    try:
        offline_empty = driver.find_element(By.CSS_SELECTOR, "picture.mr-2 + div")
        if (offline_empty != None and offline_empty.text.strip().__contains__("Tidak tersedia")):
            sales_dict["Offline Stock Status"] = "Not Available"
        else:
            raise Exception("The text is not empty text")
    except:
        try:
            offline_stock_status = driver.find_element(By.CSS_SELECTOR, "div[data-testid='store-item-0']")
            if (offline_stock_status != None):
                sales_dict["Offline Stock Status"] = "Available"
        except:
            sales_dict["Offline Stock Status"] = "Unknown"

    return sales_dict

#### Extract Customer and Review Table Function

In [46]:
def extract_customer_review(driver, wait):
    customer_dict = {
        "Email" : "",
        "Name" : "",
        "Member Status" : ""
    }
    customer_header = customer_dict.keys()
    customer_list = []

    review_dict = {
        "Product Name" : "",
        "Email" : "",
        "Date" : "",
        "Time" : "",
        "Rating" : "",
        "Delivery Review" : "",
        "Product Review" : ""
    }
    review_header = review_dict.keys()
    review_list = []

    MAX_REVIEW_PER_PRODUCT = 5
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pb-14")))
    div_review_pagination = driver.find_element(By.CSS_SELECTOR, ".pb-14")
    try:
        max_tab_per_product = int(div_review_pagination.find_elements(By.TAG_NAME, "li")[-2].text)
    except:
        max_tab_per_product = 1

    review_count = 0
    tab_count = 0
    # driver.implicitly_wait(10)
    while (review_count < MAX_REVIEW_PER_PRODUCT and tab_count < max_tab_per_product):
        time.sleep(1)
        rating_div = driver.find_elements(By.CSS_SELECTOR, ".bg-neutral-900.col-span-12")[1]
        review_divs = rating_div.find_elements(By.CSS_SELECTOR, ".p-4.relative")

        for review in review_divs:
            reviewer_name = review.find_element(By.CSS_SELECTOR, ".mb-1 .text-base").text
            if (len(reviewer_name) < 3) or (reviewer_name[1] == '*' and reviewer_name[-2] == '*'): continue

            product_name = driver.find_element(By.CSS_SELECTOR, "h1").text

            rating = len(review.find_elements(By.CSS_SELECTOR, ".rating .ic-star-fill"))
            datetime_review = to_isoformat(review.find_element(By.CSS_SELECTOR, "span.block.text-xxs").text)
            date_review, time_review = datetime_review[0], datetime_review[1]
            
            paragraph_review = review.find_elements(By.CSS_SELECTOR, "p")
            delivery_review = ""
            product_review = ""
            for par in paragraph_review:
                title = par.find_element(By.CSS_SELECTOR, "span").text
                if title == "Pengiriman:": delivery_review = par.text.lstrip("Pengiriman: ")
                if title == "Produk:": product_review = par.text.lstrip("Produk: ")
                else: continue
            if delivery_review == "": delivery_review = "Tidak ada review"
            if product_review == "": product_review = "Tidak ada review"

            member_status = driver.find_element(By.CSS_SELECTOR, ".badge.text-xxs").text

            review_count += 1
            if review_count > MAX_REVIEW_PER_PRODUCT: break

            customer_result = [to_email(reviewer_name), reviewer_name, member_status]
            for key, value in zip(customer_header, customer_result):
                customer_dict[key] = value
            customer_list.append(customer_dict.copy())
        
            result_review = [product_name, to_email(reviewer_name), date_review, time_review, rating, delivery_review, product_review]
            for key, value in zip(review_header, result_review):
                review_dict[key] = value
            review_list.append(review_dict.copy())
        
        tab_count += 1
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pb-14")))
        div_review_pagination = driver.find_element(By.CSS_SELECTOR, ".pb-14")
        page_buttons = div_review_pagination.find_elements(By.TAG_NAME, "li")
        for page_button in page_buttons:
            if page_button.text == str(tab_count + 1): 
                page_button.click()
                break

    return [customer_list, review_list]

### Main Program

In [47]:
def main():
    # instantiate the driver and the wait object
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    wait = WebDriverWait(driver, 10)

    # define the URL
    URL = "https://www.jamtangan.com/c/jam-tangan"

    # open the URL
    driver.maximize_window()
    driver.get(URL)

    try:
        # close the pop-up if it exists
        try:
            if (driver.find_element(By.ID, "driver-popover-item")):
                driver.find_element(By.CLASS_NAME, "driver-close-btn").click()

            time.sleep(3)
            if (driver.find_element(By.CSS_SELECTOR, "button.ng-binding")):
                driver.find_element(By.CSS_SELECTOR, "button.ng-binding").click()
        except:
            pass

        # expand the brands list in filter sidebar
        expand_brand = driver.find_elements(By.CSS_SELECTOR, ".accordion-content-wrapper .cursor-pointer")[1]
        while(expand_brand.text == "LIHAT LAINNYA"):
            expand_brand.click()
            time.sleep(1)
            expand_brand = driver.find_elements(By.CSS_SELECTOR, ".accordion-content-wrapper .cursor-pointer")[1]

        # get the brands name and url that eligible to be scraped
        brands_raw = driver.find_elements(By.CSS_SELECTOR, ".accordion-content-wrapper")[1].find_elements(By.TAG_NAME, "li")
        eliminate = ['Semua Brand', 'Band', 'Strap', 'Bracelet', 'Accessories', 'Jewelry', 'Wallets']
        brands = []
        for brand in brands_raw:
            eliminated = False
            for phrase in eliminate:
                if (phrase in brand.find_element(By.TAG_NAME, "label").text):
                    eliminated = True
                    break
            if not eliminated:
                brands.append(brand)
        brands = brands[39:]
        brands_name = [brand.find_element(By.TAG_NAME, "label").text for brand in brands]
        brands_url = [brand.find_element(By.TAG_NAME, "a").get_attribute("href") for brand in brands]

        # declare the container of the tables
        product = []
        sales = []
        customer = []
        review = []

        # scrape the data by iterating each product through the brands
        max_product_per_brand = 20
        product_count = 1
        for brand_url in brands_url:
            driver.get(brand_url)
            wait.until(EC.url_to_be(brand_url))
            time.sleep(1)
            products_tag = driver.find_elements(By.CSS_SELECTOR, "a[data-testid='product-card-test']")[:max_product_per_brand]
            products_url = [product.get_attribute("href") for product in products_tag]
            for product_url in products_url:
                try:
                    driver.get(product_url)
                    wait.until(EC.url_to_be(product_url))
                    print(f"{product_count}. Brand = {brands_name[brands_url.index(brand_url)]} | Product = {product_url}")
                    extracted_product = extract_product(driver, wait)
                    extracted_sales = extract_sales(driver, wait)
                    extracted_customer = extract_customer_review(driver, wait)[0]
                    extracted_review = extract_customer_review(driver, wait)[1]

                    product.append(extracted_product)
                    sales.append(extracted_sales)
                    for item in extracted_customer: 
                        if (item not in customer):
                            customer.append(item)
                    for item in extracted_review: review.append(item)
                    product_count += 1
                except:
                    traceback.print_exc()
                    continue
        driver.close()  
    except:
        traceback.print_exc()
        pass        

    try:
        save_dir = r'../data/unprocessed/'

        # exporting the product table to csv and json
        with open(save_dir + "product.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=product[0].keys())
            writer.writeheader()
            writer.writerows(product)
        product_csv_file_path = save_dir + r'product.csv'
        product_json_file_path = save_dir + r'product.json'
        csv_to_json(product_csv_file_path, product_json_file_path)

        # exporting the sales table to csv and json
        with open(save_dir + "sales.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=sales[0].keys())
            writer.writeheader()
            writer.writerows(sales)
        sales_csv_file_path = save_dir + r'sales.csv'
        sales_json_file_path = save_dir + r'sales.json'
        csv_to_json(sales_csv_file_path, sales_json_file_path)

        # exporting the customer table to csv and json
        with open(save_dir + "customer.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=customer[0].keys())
            writer.writeheader()
            writer.writerows(customer)
        customer_csv_file_path = save_dir + r'customer.csv'
        customer_json_file_path = save_dir + r'customer.json'
        csv_to_json(customer_csv_file_path, customer_json_file_path)

        # exporting the review table to csv and json
        with open(save_dir + "review.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=review[0].keys())
            writer.writeheader()
            writer.writerows(review)
        review_csv_file_path = save_dir + r'review.csv'
        review_json_file_path = save_dir + r'review.json'
        csv_to_json(review_csv_file_path, review_json_file_path)

    except:
        traceback.print_exc()
        pass
    
    print(f"Product Collected = {len(product)}")
    print(f"Sales Collected = {len(sales)}")
    print(f"Customer Collected = {len(customer)}")
    print(f"Review Collected = {len(review)}")

    print("\n========== Scraping is done! ==========\n")

In [3]:
if (__name__ == "__main__"):
    # setup the timer
    time_start = datetime.now()

    # run the main program
    main()

    # setup the timer
    time_end = datetime.now()
    delta = time_end - time_start

    text_wrapper = open(r"Data Scraping\src\config\email_config.txt", "r")

    email_config = text_wrapper.readlines()
    email_to_notify = email_config[0].strip()
    email_password = email_config[1].strip()

    text_wrapper.close()

    subject = "Scraping Jamtangan.com Report"
    body = f"""
    [SCRAPING JAMTANGAN.COM IS COMPLETE]

    Time Start:\t{time_start.strftime('%Y-%m-%d %H:%M:%S')}
    Time End:\t{time_end.strftime('%Y-%m-%d %H:%M:%S')}
    Duration:\t{str(delta).split('.')[0]}

    """

    email = EmailMessage()
    email["From"] = email_to_notify
    email["To"] = email_to_notify
    email["Subject"] = subject
    email.set_content(body)

    context = ssl.create_default_context()

    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(email_to_notify, email_password)
        server.sendmail(email_to_notify, email_to_notify, email.as_string())
    
    print(f"Time Start:\t{time_start.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Time End:\t{time_end.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Duration:\t{str(delta).split('.')[0]}\n")


Product Collected = 692
Sales Collected = 692
Customer Collected = 1325
Review Collected = 1422

========== Scraping is done! ==========

Time Start:	2023-07-18 16:28:13
Time End:	2023-07-18 18:24:43
Duration:	1:56:30  
    


### Additional Pre-Processing

#### Pre-Processing Data Product

In [97]:
# load the product data
unprocessed_product_df = pd.read_json(r"../data/unprocessed/product.json")

# check duplicate data
duplicated_product_stat = unprocessed_product_df.duplicated()
num_duplicated_product = duplicated_product_stat.sum()

print(f"===== Jumlah data product yang duplikat: {num_duplicated_product}")

# check and handle bad product
print(f"===== Jumlah raw data product include bad product: {len(unprocessed_product_df)}")
missing_amount = unprocessed_product_df.eq("").sum(axis=1)
filtered_product_df = unprocessed_product_df[missing_amount <= 2]
print(f"===== Contoh bad product: \n{unprocessed_product_df[missing_amount > 2].head(3)}")
print(f"===== Jumlah raw data product setelah filtering bad product: {len(filtered_product_df)}")

# check and handle missing value
missing_value = (filtered_product_df == "").sum().sum()
print(f"===== Jumlah data product yang memiliki nilai kosong: {missing_value}")

filtered_product_df = filtered_product_df.replace("", "-")
missing_value = (filtered_product_df == "").sum().sum()
print(f"===== Jumlah data product yang memiliki nilai kosong setelah dihandle: {missing_value}")

# export the filtered product data
file_path = r"../data/processed/product.json"
df_json = filtered_product_df.to_json(file_path, orient="records")

===== Jumlah data product yang duplikat: 0
===== Jumlah raw data product include bad product: 1339
===== Contoh bad product: 
            Brand               Model No    Series Gender Colour Luminous  \
173  Boda Concept  BC-C1-CF-CARBON-FIBER  C Series                          
402           DLW   HANDS-MIL-SPEC-BLACK                                    
403           DLW    FLAT-BI-CI-SKX-BLUE                                    

    Calendar Water Resistant Case Diameter (mm) Strap Material  
173                                                             
402                                                             
403                                                             
===== Jumlah raw data product setelah filtering bad product: 1252
===== Jumlah data product yang memiliki nilai kosong: 647
===== Jumlah data product yang memiliki nilai kosong setelah dihandle: 0


#### Pre-Processing Data Sales

In [98]:
# load the sales data
unprocessed_sales_df = pd.read_json(r"../data/unprocessed/sales.json")

# check duplicate data
duplicated_sales_stat = unprocessed_sales_df.duplicated()
num_duplicated_sales = duplicated_sales_stat.sum()

print(f"===== Jumlah data sales yang duplikat: {num_duplicated_sales}")

# remove sales that include bad product
print(f"===== Jumlah raw data sales include bad product: {len(unprocessed_sales_df)}")
filtered_sales_df = unprocessed_sales_df[
    unprocessed_sales_df["Brand"].isin(filtered_product_df["Brand"]) & 
    unprocessed_sales_df["Model No"].isin(filtered_product_df["Model No"])]
print(f"===== Jumlah raw data sales setelah filtering bad product: {len(filtered_sales_df)}")

# export the filtered sales data
file_path = r"../data/processed/sales.json"
df_json = filtered_sales_df.to_json(file_path, orient="records")

===== Jumlah data sales yang duplikat: 0
===== Jumlah raw data sales include bad product: 1339
===== Jumlah raw data sales setelah filtering bad product: 1252


#### Pre-Processing Data Review

In [99]:
# load the review data
unprocessed_review_df = pd.read_json(r"../data/unprocessed/review.json")

# remove review that include bad product
print(f"===== Jumlah raw data review include bad product: {len(unprocessed_review_df)}")
filtered_review_df = unprocessed_review_df[unprocessed_review_df['Product Name'].isin(filtered_sales_df['Product Name'])]
print(f"===== Jumlah raw data review setelah filtering bad product: {len(filtered_review_df)}")

# check and handle duplicate data
duplicated_review_stat = filtered_review_df.duplicated()
num_duplicated_review = duplicated_review_stat.sum()
duplicated_review_rows = filtered_review_df[duplicated_review_stat]
duplicateless_review = filtered_review_df.drop_duplicates()

print(f"===== Jumlah data review yang duplikat: {num_duplicated_review}")
print(f"===== Data yang duplikat:\n{duplicated_review_rows}")
print(f"===== Jumlah data review yang duplikat setelah diproses: {duplicateless_review.duplicated().sum()}")

# export the filtered review data
file_path = r"../data/processed/review.json"
df_json = duplicateless_review.to_json(file_path, orient="records")

===== Jumlah raw data review include bad product: 2650
===== Jumlah raw data review setelah filtering bad product: 2496
===== Jumlah data review yang duplikat: 437
===== Data yang duplikat:
                                           Product Name  \
17    Alba Prestige ARSY96X1 Men Silver Dial Dual To...   
57    Alba Prestige AJ6124X1 Men Silver Dial Dual To...   
95    Alexandre Christie Chronograph AC 6141 MC BTBB...   
96    Alexandre Christie Chronograph AC 6141 MC BTBB...   
97    Alexandre Christie Chronograph AC 6141 MC BTBB...   
...                                                 ...   
2590  TISSOT T-Sport T116.617.11.047.01 Chrono XL Cl...   
2592  TISSOT Luxury Powermatic 80 T086.407.11.201.02...   
2594  TISSOT T-Sport T116.617.11.057.01 Chrono XL Cl...   
2596  Tissot PR 100 Chronograph Gent White Dial Stai...   
2598  Tissot T-Classic T109.407.16.031.00 Everytime ...   

                            Email        Date      Time  Rating  \
17         nadhimfaisal@gmail.com 

#### Pre-Processing Data Customer

In [100]:
# load the customer data
unprocessed_customer_df = pd.read_json(r"../data/unprocessed/customer.json")

# check and handle duplicate data
duplicated_cust_stat = unprocessed_customer_df.duplicated(subset=['Email'])
num_duplicated_cust = duplicated_cust_stat.sum()
print(f"===== Jumlah data customer yang duplikat (email yang sama): {num_duplicated_cust}")

print("===== Contoh data dengan email yang sama dan member status yang bisa berbeda:")
df_cust_email_sorted = unprocessed_customer_df.sort_values(by='Email')
duplicated_cust_stat = df_cust_email_sorted.duplicated(subset='Email', keep=False)
duplicated_cust_rows = df_cust_email_sorted[duplicated_cust_stat]
print(duplicated_cust_rows[:6])

# delete duplicated rows with same email and keep the highest member status
duplicateless_cust = df_cust_email_sorted.drop_duplicates(subset='Email', keep='first')

print(f"\n===== Jumlah data customer yang duplikat setelah diproses: {duplicateless_cust.duplicated(subset=['Email']).sum()}")

# export the filtered customer data
file_path = r"../data/processed/customer.json"
df_json = duplicateless_cust.to_json(file_path, orient="records")

===== Jumlah data customer yang duplikat (email yang sama): 229
===== Contoh data dengan email yang sama dan member status yang bisa berbeda:
                    Email        Name     Member Status
1653  abdulazis@gmail.com  Abdul azis  Centurion Member
1998  abdulazis@gmail.com  Abdul azis      Basic Member
764         ade@gmail.com         Ade     Silver Member
2195        ade@gmail.com         Ade      Basic Member
1012      agung@gmail.com       agung      Basic Member
206       agung@gmail.com       Agung      Basic Member

===== Jumlah data customer yang duplikat setelah diproses: 0
